# Diffusion Limited Aggregation 

#### Author : B. Militzer, University of California, Berkeley 
#### Date   : Sept. 26, 2018

#### Read "The Science of Fractal Images", Ed. Peitgen and Saupe, p. 37 (1988)


In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#note, this function expects a matrix A[ix,iy] 
#and then displays so that A[:,0] is the lowest row of pixels
def display(A):
    maxX = A.shape[0]
    maxY = A.shape[1]
    B = np.zeros((maxY, maxX))
    for ix in range(0,maxX):
        for iy in range(0,maxY):
            B[maxY-1-iy,ix] = A[ix,iy]

    #Display the graphics outside of the notebook. 
    #On a PC, use '%matplotlib qt' instead.
    %matplotlib tk
    #NOTE: I am running Ubuntu and deleting this line worked for me. (having that line caused errors.)
    
    plt.rcParams['figure.figsize'] = [6, 6/maxX*maxY]
    plt.imshow(B); 
    plt.axis('off'); 
    plt.show()
    plt.draw()
    plt.pause(0.01)

In [29]:
nParticles = 10000
maxX = 300
maxY = 200

In [30]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

# Introduce a sticky wall at the bottom 
# by filling the lowest row of pixels with particles
A[:,0] = 1
print(A.transpose())

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
# To save computer time, we want to inject the new particle not too far
# above growing aggregate. We inject at on a line 'yStart', which
# keeps being increased so that it is always 'yBuffer' lines above the
# highest structure
yBuffer = 5
yStart  = 1 + yBuffer

In [32]:
p = 0.05

In [33]:
for i in range(0,nParticles):
    # Compute new starting point on the line y=yStart
    x  = np.random.randint(0,maxX)
    y  = yStart; #always start at upper limit

    while True:
        xOrg = x
        yOrg = y

        r = np.random.random(); # Random float:  0.0 <= r < 1.0
        #based on the value of 'r', move the particle
        #left, right, up, or down and change x and y accordingly
        if r < 0.25: 
            x -= 1
        elif 0.25 <= r < 0.5: 
            x +=1
        elif 0.5 <= r < 0.75: 
            y += 1
        else: 
            y -= 1
        
        #now apply periodic boundary conditions to 'x'
        if y < 0: 
            y = maxY-1
        elif y >= maxY:
            y = 0
        if x < 0:
            x = maxX-1
        elif x >= maxX:
            x = 0
        
        
        
        if (A[x,y] == 1 or y>yStart): 
            x = xOrg
            y = yOrg
            continue; # if this site has been taken try moving in a different direction
        
        #determine the x coordionates of the left and right neighbors
        #store them in 'xm' and 'xp' and apply periodic boundary conditions again
        xp = x+1
        xm = x-1
        yp = y+1
        ym = y-1
        if ym < 0:
            ym = maxY-1
        elif yp >= maxY:
            yp = 0
        if xm < 0:
            xm = maxX-1
        elif xp >= maxX:
            xp = 0
 
        # Determine if any neighboring site is occupied
        # if that is the case, enter the following 'if' clause
        r1 = np.random.random();
        if r1 <= p:
            if (y>yStart or A[xp,y] == 1 or A[xm,y] == 1 or A[x,yp] == 1 or A[x,ym] == 1): 
                A[x,y] = 1
                if (y+yBuffer>yStart and y+yBuffer<maxY): 
                    yStart = y+yBuffer

                if (i%1000==0): 
                    print(f'i= {i} \tx={x} \ty={y} \tyStart={yStart}')

                nNewParticlesPerFrame = 1000 
                if (i%nNewParticlesPerFrame==0): 
                    display(A)

                break # particle was attached, break out of current loop and insert next one
            
    if (yStart+1==maxY): 
        print(f'Structures reached Y limit after only {i} particles')
        break

display(A)
        

i= 0 	x=37 	y=1 	yStart=6
i= 1000 	x=205 	y=4 	yStart=17
i= 2000 	x=48 	y=15 	yStart=24
i= 3000 	x=229 	y=16 	yStart=34
i= 4000 	x=269 	y=39 	yStart=44
i= 5000 	x=188 	y=38 	yStart=53
i= 6000 	x=192 	y=53 	yStart=63
i= 7000 	x=183 	y=57 	yStart=73
i= 8000 	x=53 	y=69 	yStart=81
i= 9000 	x=165 	y=73 	yStart=94


# Part II:
The They grow with the stream (like stalactites)